In [8]:
!pip install gym

# Import required libraries
import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
import glob
import io
from base64 import b64encode
from gym.wrappers import RecordVideo

In [10]:
def discretize_state(state, bins):
    position_bins = np.linspace(-1.2, 0.6, bins)  # Discretize position
    velocity_bins = np.linspace(-0.07, 0.07, bins)  # Discretize velocity
    position_idx = np.digitize(state[0], position_bins) - 1
    velocity_idx = np.digitize(state[1], velocity_bins) - 1
    return (position_idx, velocity_idx)

# Function to display video
def show_video():
    video_path = glob.glob('./video/*.mp4')[0]
    video = io.open(video_path, 'r+b').read()
    encoded = b64encode(video)
    return HTML(data=f'''
        ''')

# Initialize the environment
env = gym.make("MountainCar-v0")

# Hyperparameters
n_bins = 20  # Number of bins for discretization
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 1.0  # Exploration rate
epsilon_decay = 0.995
epsilon_min = 0.01
n_episodes = 5000

# Initialize Q-table
n_actions = env.action_space.n
q_table = np.zeros((n_bins, n_bins, n_actions))

# Training loop
for episode in range(n_episodes):
    state = discretize_state(env.reset(), n_bins)
    done = False
    total_reward = 0

    while not done:
        # Choose action: exploration vs exploitation
        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])

        # Perform action
        next_state_raw, reward, done, _ = env.step(action)
        next_state = discretize_state(next_state_raw, n_bins)

        # Update Q-value
        best_next_action = np.argmax(q_table[next_state])
        td_target = reward + gamma * q_table[next_state][best_next_action]
        q_table[state][action] += alpha * (td_target - q_table[state][action])

        state = next_state
        total_reward += reward

    # Decay epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    if episode % 100 == 0:
        print(f"Episode {episode}, Total Reward: {total_reward}")

print("Training Complete!")

# Wrap environment for recording
env = RecordVideo(env, "./video", episode_trigger=lambda x: True)

# Test the trained agent
state = discretize_state(env.reset(), n_bins)
done = False
while not done:
    action = np.argmax(q_table[state])
    next_state_raw, _, done, _ = env.step(action)
    state = discretize_state(next_state_raw, n_bins)

env.close()

# Display the video
show_video()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 0, Total Reward: -200.0
Episode 100, Total Reward: -200.0
Episode 200, Total Reward: -200.0
Episode 300, Total Reward: -200.0
Episode 400, Total Reward: -200.0
Episode 500, Total Reward: -200.0
Episode 600, Total Reward: -200.0
Episode 700, Total Reward: -200.0
Episode 800, Total Reward: -200.0
Episode 900, Total Reward: -200.0
Episode 1000, Total Reward: -200.0
Episode 1100, Total Reward: -200.0
Episode 1200, Total Reward: -200.0
Episode 1300, Total Reward: -200.0
Episode 1400, Total Reward: -159.0
Episode 1500, Total Reward: -162.0
Episode 1600, Total Reward: -191.0
Episode 1700, Total Reward: -200.0
Episode 1800, Total Reward: -200.0
Episode 1900, Total Reward: -164.0
Episode 2000, Total Reward: -200.0
Episode 2100, Total Reward: -200.0
Episode 2200, Total Reward: -160.0
Episode 2300, Total Reward: -151.0
Episode 2400, Total Reward: -170.0
Episode 2500, Total Reward: -150.0
Episode 2600, Total Reward: -194.0
Episode 2700, Total Reward: -200.0
Episode 2800, Total Reward: -155

/usr/local/lib/python3.10/dist-packages/gym/wrappers/record_video.py:78: UserWarning: WARN: Overwriting existing videos at /content/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment MountainCar-v0 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
